# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## NMI

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Constipation
df = pd.read_csv('NMI_Top50_Constipation_Methylome_Female.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Constipation
(150, 52)


,PATNO,APPRDX,cg14268784,cg22136726,cg00335286,cg20080383,cg17594730,cg03166896,cg21409615,cg09351848,...,cg19402876,cg07857267,cg06838963,cg15657588,cg20170632,cg12103037,cg00140813,cg19238390,cg18862481,cg04152151
0,3002,1,0.752473,0.699776,0.650353,0.644179,0.905597,0.941780,0.882367,0.864516,...,0.908583,0.891956,0.828953,0.475535,0.863334,0.088004,0.023514,0.822136,0.016866,0.877233
1,3003,1,0.742175,0.667122,0.708811,0.461084,0.937221,0.937959,0.893561,0.855371,...,0.932988,0.913486,0.819469,0.490711,0.756304,0.079265,0.013589,0.846568,0.015620,0.779324
2,3006,1,0.719903,0.680338,0.659652,0.725966,0.935031,0.930896,0.900978,0.842452,...,0.943975,0.901019,0.865209,0.479025,0.829658,0.064628,0.014707,0.814955,0.019325,0.830126


In [3]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg14268784,cg22136726,cg00335286,cg20080383,cg17594730,cg03166896,cg21409615,cg09351848,cg01493658,...,cg19402876,cg07857267,cg06838963,cg15657588,cg20170632,cg12103037,cg00140813,cg19238390,cg18862481,cg04152151
0,1,0.752473,0.699776,0.650353,0.644179,0.905597,0.941780,0.882367,0.864516,0.024975,...,0.908583,0.891956,0.828953,0.475535,0.863334,0.088004,0.023514,0.822136,0.016866,0.877233
1,1,0.742175,0.667122,0.708811,0.461084,0.937221,0.937959,0.893561,0.855371,0.018995,...,0.932988,0.913486,0.819469,0.490711,0.756304,0.079265,0.013589,0.846568,0.015620,0.779324


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
print(dframe1.head(2))
dframe1.to_csv ("Constipation_NMICpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.662857  0.173229
1  0.737143  0.154814


## Boruta

In [6]:
df = pd.read_csv('Boruta_Top50_Constipation_Methylome_Female.csv')
print(df.shape)
df.head(3)

(150, 52)


,PATNO,APPRDX,cg15995909,cg26950867,cg07198258,cg12103037,cg20383686,cg11712482,cg00265238,cg23814714,...,cg09896120,cg25734490,cg01801174,cg17474357,cg13332222,cg18322516,cg04979551,cg24926711,cg02308280,cg19113530
0,3002,1,0.912166,0.756753,0.790367,0.088004,0.043670,0.426216,0.036745,0.879547,...,0.053850,0.024795,0.020359,0.670732,0.885483,0.922824,0.926786,0.181981,0.659316,0.053468
1,3003,1,0.923107,0.657526,0.802884,0.079265,0.056042,0.431001,0.037143,0.900867,...,0.035119,0.022206,0.037418,0.704347,0.945536,0.912891,0.913626,0.182101,0.705500,0.052371
2,3006,1,0.930580,0.689543,0.835179,0.064628,0.042821,0.444690,0.025035,0.942837,...,0.032386,0.022256,0.019968,0.708061,0.940464,0.944668,0.922240,0.152496,0.746687,0.049015


In [7]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg15995909,cg26950867,cg07198258,cg12103037,cg20383686,cg11712482,cg00265238,cg23814714,cg23446297,...,cg09896120,cg25734490,cg01801174,cg17474357,cg13332222,cg18322516,cg04979551,cg24926711,cg02308280,cg19113530
0,1,0.912166,0.756753,0.790367,0.088004,0.043670,0.426216,0.036745,0.879547,0.053269,...,0.053850,0.024795,0.020359,0.670732,0.885483,0.922824,0.926786,0.181981,0.659316,0.053468
1,1,0.923107,0.657526,0.802884,0.079265,0.056042,0.431001,0.037143,0.900867,0.079942,...,0.035119,0.022206,0.037418,0.704347,0.945536,0.912891,0.913626,0.182101,0.705500,0.052371


In [8]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
print(dframe1.head(2))
dframe1.to_csv ("Constipation_BorutaCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.731429  0.168159
1  0.757143  0.172023


## BorutaR

In [10]:
df = pd.read_csv('BorutaR_Constipation_CpG_Female.csv')
print(df.shape)
df.head(3)

(150, 11)


,PATNO,APPRDX,cg19930203,cg09407580,cg12671744,cg26950867,cg22597771,cg23446297,cg24926711,cg24305835,cg19458274
0,3002,1,0.836213,0.797046,0.600627,0.756753,0.574155,0.053269,0.181981,0.043436,0.015144
1,3003,1,0.824967,0.762141,0.503826,0.657526,0.544363,0.079942,0.182101,0.018344,0.015374
2,3006,1,0.833875,0.741682,0.514428,0.689543,0.555292,0.054852,0.152496,0.022896,0.017079


In [11]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg19930203,cg09407580,cg12671744,cg26950867,cg22597771,cg23446297,cg24926711,cg24305835,cg19458274
0,1,0.836213,0.797046,0.600627,0.756753,0.574155,0.053269,0.181981,0.043436,0.015144
1,1,0.824967,0.762141,0.503826,0.657526,0.544363,0.079942,0.182101,0.018344,0.015374


In [12]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
print(dframe1.head(2))
dframe1.to_csv ("Constipation_BorutaRCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.694286  0.180724
1  0.651429  0.166990


## Common

In [14]:
df = pd.read_csv('Common_Constipation_CpG_Female.csv')
print(df.shape)
df.head(3)

(150, 10)


,PATNO,APPRDX,cg26950867,cg23446297,cg24926711,cg24305835,cg19930203,cg12103037,cg05896001,cg16694455
0,3002,1,0.756753,0.053269,0.181981,0.043436,0.836213,0.088004,0.857006,0.907496
1,3003,1,0.657526,0.079942,0.182101,0.018344,0.824967,0.079265,0.788452,0.908508
2,3006,1,0.689543,0.054852,0.152496,0.022896,0.833875,0.064628,0.849319,0.889825


In [15]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg26950867,cg23446297,cg24926711,cg24305835,cg19930203,cg12103037,cg05896001,cg16694455
0,1,0.756753,0.053269,0.181981,0.043436,0.836213,0.088004,0.857006,0.907496
1,1,0.657526,0.079942,0.182101,0.018344,0.824967,0.079265,0.788452,0.908508


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
print(dframe1.head(2))
dframe1.to_csv ("Constipation_CommonCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.742857  0.139971
1  0.731429  0.182141
